# Query Pipeline for Advanced Text-to-SQL¶

In this guide we show you how to setup a text-to-SQL pipeline over your data with our query pipeline syntax.

This gives you flexibility to enhance text-to-SQL with additional techniques. We show these in the below sections:

1. Query-Time Table Retrieval: Dynamically retrieve relevant tables in the text-to-SQL prompt.
2. Query-Time Sample Row retrieval: Embed/Index each row, and dynamically retrieve example rows for each table in the text-to-SQL prompt.
   Our out-of-the box pipelines include our NLSQLTableQueryEngine and SQLTableRetrieverQueryEngine. (if you want to check out our text-to-SQL guide using these modules, take a look here). This guide implements an advanced version of those modules, giving you the utmost flexibility to apply this to your own setting.

NOTE: Any Text-to-SQL application should be aware that executing arbitrary SQL queries can be a security risk. It is recommended to take precautions as needed, such as using restricted roles, read-only databases, sandboxing, etc.

## Load and Ingest Data

### Load Data

We use the [WikiTableQuestions](https://github.com/ppasupat/WikiTableQuestions/releases) dataset (Pasupat and Liang 2015) as our test dataset.

We go through all the csv's in one folder, store each in a sqlite database (we will then build an object index over each table schema).


In [9]:
import pandas as pd
from pathlib import Path

data_dir = Path("./WikiTableQuestions/csv/200-csv")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: WikiTableQuestions/csv/200-csv/0.csv
processing file: WikiTableQuestions/csv/200-csv/1.csv
processing file: WikiTableQuestions/csv/200-csv/10.csv
processing file: WikiTableQuestions/csv/200-csv/11.csv
processing file: WikiTableQuestions/csv/200-csv/12.csv
processing file: WikiTableQuestions/csv/200-csv/14.csv
processing file: WikiTableQuestions/csv/200-csv/15.csv
Error parsing WikiTableQuestions/csv/200-csv/15.csv: Error tokenizing data. C error: Expected 4 fields in line 16, saw 5

processing file: WikiTableQuestions/csv/200-csv/17.csv
Error parsing WikiTableQuestions/csv/200-csv/17.csv: Error tokenizing data. C error: Expected 6 fields in line 5, saw 7

processing file: WikiTableQuestions/csv/200-csv/18.csv
processing file: WikiTableQuestions/csv/200-csv/20.csv
processing file: WikiTableQuestions/csv/200-csv/22.csv
processing file: WikiTableQuestions/csv/200-csv/24.csv
processing file: WikiTableQuestions/csv/200-csv/25.csv
processing file: WikiTableQuestions/csv/200-

### Extract Table Name and Summary from each Table

Here we use gpt-3.5 to extract a table name (with underscores) and summary from each table with our Pydantic program.


In [ ]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise. 
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str=prompt_str,
)

print(program)

In [ ]:
import json


def _get_tableinfo_with_index(idx: int):
    results_gen = Path("WikiTableQuestions_TableInfo").glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    if len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"WikiTableQuestions_TableInfo/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

### Put Data in SQL Database

We use sqlalchemy, a popular SQL database toolkit, to load all the tables.


In [ ]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re


# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

Setup Arize Phoenix for observability


In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"  # Phoenix receiver address

tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(
    SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

## Advanced Capability 1: Text-to-SQL with Query-Time Table Retrieval.

We now show you how to setup an e2e text-to-SQL with table retrieval.

Here we define the core modules.

1. Object index + retriever to store table schemas
2. SQLDatabase object to connect to the above tables + SQLRetriever.
3. Text-to-SQL Prompt
4. Response synthesis Prompt
5. LLM

### 1. Object index, retriever, SQLDatabase


In [ ]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(objects=table_schema_objs,
                                     object_mapping=table_node_mapping,
                                     index_cls=VectorStoreIndex,
                                     )
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

### 2. SQLRetriever + Table Parser


In [ ]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

### 3. Text-to-SQL Prompt + Output Parser


In [ ]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse


def extract_sql_query(content: str) -> str:
    sql_query_start = content.find("SQLQuery:")
    if sql_query_start == -1:
        raise ValueError("No 'SQLQuery:' marker found in the response content")

    query_content = content[sql_query_start + len("SQLQuery:"):]
    sql_result_start = query_content.find("SQLResult:")

    if sql_result_start != -1:
        query_content = query_content[:sql_result_start]

    return query_content


def clean_sql_query(query: str) -> str:
    return query.strip().strip("```").strip()


def parse_response_to_sql(response: ChatResponse) -> str:
    """
    Parse a ChatResponse object to extract the SQL query.

    This function takes a ChatResponse object, which is expected to contain
    an SQL query within its content, and extracts the SQL query string.
    The function looks for specific markers ('SQLQuery:' and 'SQLResult:')
    to identify the SQL query portion of the response.

    Args:
        response (ChatResponse): A ChatResponse object containing the response
                                 from a text-to-SQL model.

    Returns:
        str: The extracted SQL query as a string, with surrounding whitespace
             and code block markers (```) removed.

    Raises:
        AttributeError: If the input doesn't have the expected 'message.content' attribute.
        ValueError: If no 'SQLQuery:' marker is found in the response content.

    Note:
        - The function assumes that the SQL query is preceded by 'SQLQuery:' 
          and optionally followed by 'SQLResult:'.
        - Any content before 'SQLQuery:' or after 'SQLResult:' is discarded.
        - The function removes leading/trailing whitespace and code block markers.

    Example:
        >>> response = ChatResponse(message=Message(content="Some text\nSQLQuery: SELECT * FROM table\nSQLResult: ..."))
        >>> sql_query = parse_response_to_sql(response)
        >>> print(sql_query)
        SELECT * FROM table
    """
    try:
        content = str(response.message.content)
    except AttributeError:
        raise ValueError(
            "Input must be a ChatResponse object with a 'message.content' attribute")

    sql_query = extract_sql_query(content)
    return clean_sql_query(sql_query)


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

### 4. Response Synthesis Prompt


In [ ]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

### 5. LLM


In [ ]:
llm = OpenAI(model="gpt-3.5-turbo")

#### Define Query Pipeline

Now that the components are in place, let's define the query pipeline!


In [ ]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)
qp

In [ ]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

#### Visualize Query Pipeline

A really nice property of the query pipeline syntax is you can easily visualize it in a graph via networkx.


In [ ]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)

In [ ]:
# Save the network as "text2sql_dag.html"
net.write_html("text2sql_dag.html")

In [ ]:
from IPython.display import display, HTML

# Read the contents of the HTML file
with open("text2sql_dag.html", "r") as file:
    html_content = file.read()

# Display the HTML content
display(HTML(html_content))

### Run Some Queries!

Now we're ready to run some queries across this entire pipeline.


In [ ]:
response = qp.run(
    query="What was the year that The Notorious B.I.G was signed to Bad Boy?"
)
print(str(response))

In [ ]:
response = qp.run(query="Who won best director in the 1972 academy awards")
print(str(response))

## Advanced Capability 2: Text-to-SQL with Query-Time Row Retrieval (along with Table Retrieval)

One problem in the previous example is that if the user asks a query that asks for "The Notorious BIG" but the artist is stored as "The Notorious B.I.G", then the generated SELECT statement will likely not return any matches.

We can alleviate this problem by fetching a small number of example rows per table. A naive option would be to just take the first k rows. Instead, we embed, index, and retrieve k relevant rows given the user query to give the text-to-SQL LLM the most contextually relevant information for SQL generation.

We now extend our query pipeline.

## Index Each Table

We embed/index the rows of each table, resulting in one index per table.


In [ ]:
import logging
from pathlib import Path
from typing import Dict, Optional
from llama_index.core import VectorStoreIndex, load_index_from_storage
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import text

logger = logging.getLogger(__name__)


def get_table_rows(engine, table_name: str):
    try:
        with engine.connect() as conn:
            cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
            return [tuple(row) for row in cursor.fetchall()]
    except SQLAlchemyError as e:
        logger.error(f"Error fetching rows from table {table_name}: {str(e)}")
        raise


def create_index(rows, index_path: Path):
    nodes = [TextNode(text=str(t)) for t in rows]
    index = VectorStoreIndex(nodes)
    index.set_index_id("vector_index")
    index.storage_context.persist(str(index_path))
    return index


def load_existing_index(index_path: Path):
    storage_context = StorageContext.from_defaults(persist_dir=str(index_path))
    return load_index_from_storage(storage_context, index_id="vector_index")


def index_all_tables(
    sql_database,
    table_index_dir: str = "table_index_dir",
    force_refresh: bool = False,
    tables_to_index: Optional[list] = None
) -> Dict[str, VectorStoreIndex]:
    """
    Create or load vector store indexes for specified tables in the given SQL database.

    Args:
        sql_database: An instance of SQLDatabase containing the tables to be indexed.
        table_index_dir (str): The directory where the indexes will be stored.
        force_refresh (bool): If True, recreate all indexes even if they already exist.
        tables_to_index (Optional[list]): List of table names to index. If None, index all usable tables.

    Returns:
        Dict[str, VectorStoreIndex]: A dictionary of table names to their VectorStoreIndex objects.

    Raises:
        OSError: If there's an error creating or accessing the table_index_dir.
        SQLAlchemyError: If there's an error connecting to the database or executing SQL queries.
    """
    index_dir = Path(table_index_dir)
    index_dir.mkdir(parents=True, exist_ok=True)

    vector_index_dict = {}
    tables = tables_to_index or sql_database.get_usable_table_names()

    for table_name in tables:
        index_path = index_dir / table_name
        logger.info(f"Processing table: {table_name}")

        try:
            if not index_path.exists() or force_refresh:
                logger.info(f"Creating new index for table: {table_name}")
                rows = get_table_rows(sql_database.engine, table_name)
                index = create_index(rows, index_path)
            else:
                logger.info(f"Loading existing index for table: {table_name}")
                index = load_existing_index(index_path)

            vector_index_dict[table_name] = index

        except (OSError, SQLAlchemyError) as e:
            logger.error(f"Error processing table {table_name}: {str(e)}")
            # Decide whether to continue with other tables or raise the exception

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

In [ ]:
test_retriever = vector_index_dict["Bad_Boy_Artists"].as_retriever(
    similarity_top_k=1
)
nodes = test_retriever.retrieve("P. Diddy")
print(nodes[0].get_content())

### Define Expanded Table Parser Component

We expand the capability of our table_parser_component to not only return the relevant table schemas, but also return relevant rows per table schema.

It now takes in both table_schema_objs (output of table retriever), but also the original query_str which will then be used for vector retrieval of relevant rows.


In [ ]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

### Define Expanded Query Pipeline

This looks similar to the query pipeline in section 1, but with an upgraded table_parser_component.


In [ ]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)
qp

In [ ]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

### Run Some Queries

We can now ask about relevant entries even if it doesn't exactly match the entry in the database.


In [ ]:
response = qp.run(
    query="What was the year that The Notorious BIG was signed to Bad Boy?"
)
print(str(response))